In [4]:
import numpy as np

from lightfm.datasets import fetch_movielens
import implicit
import implicit.evaluation
lightfm.cross_validation

movielens = fetch_movielens()

In [5]:
for key, value in movielens.items():
    print(key, type(value), value.shape)

train <class 'scipy.sparse.coo.coo_matrix'> (943, 1682)
test <class 'scipy.sparse.coo.coo_matrix'> (943, 1682)
item_features <class 'scipy.sparse.csr.csr_matrix'> (1682, 1682)
item_feature_labels <class 'numpy.ndarray'> (1682,)
item_labels <class 'numpy.ndarray'> (1682,)


In [19]:
train = movielens['train']
test = movielens['test']
train.shape

(943, 1682)

In [7]:
possible_interactions = train.shape[0]*train.shape[1]
interacted = len(train.nonzero()[0])
sparsity = 1 - interacted/possible_interactions
sparsitya

0.9428986095682184

In [7]:
model = implicit.als.AlternatingLeastSquares()
model.fit(train)

  0%|          | 0/15 [00:00<?, ?it/s]

In [18]:
k=200
print("P@K",implicit.evaluation.precision_at_k(model, train, test, show_progress = False, K=k))
print("NDCG@K",implicit.evaluation.ndcg_at_k(model, train, test, show_progress = False, K=k))
print("AUC@K",implicit.evaluation.AUC_at_k(model, train, test, show_progress = False, K=k))
print("MAP@K",implicit.evaluation.mean_average_precision_at_k(model, train, test, show_progress = False, K=k))

P@K 0.7254506892895016
NDCG@K 0.44311603648316505
AUC@K 0.8275031141069679
MAP@K 0.1803858599327022


In [4]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

model = LightFM(learning_rate=0.05, loss='bpr')
model.fit(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.59, test 0.20.
AUC: train 0.90, test 0.88.


In [5]:
model = LightFM(learning_rate=0.05, loss='warp')

model.fit_partial(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.61, test 0.22.
AUC: train 0.94, test 0.93.
